<a href="https://colab.research.google.com/github/LohithK00/University_Admission_Bot/blob/main/University_Admission_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q transformers sentencepiece torch


In [12]:
from transformers import pipeline

llm = pipeline(
    task="text-generation",
    model="google/flan-t5-small",
    device=-1,        # force CPU (avoids GPU/CUDA errors)
    max_length=150
)

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCa

In [13]:
def llm_response(prompt):
    response = llm(prompt)[0]["generated_text"]
    return response

In [14]:
def load_university_data():
    return {
        "university_name": "JAI University",
        "engineering_courses": {
            "B.Tech Computer Science": 120000,
            "B.Tech Artificial Intelligence": 130000,
            "B.Tech Data Science": 125000,
            "B.Tech Cyber Security": 128000,
            "B.Tech Electronics": 110000,
            "B.Tech Mechanical": 105000,
            "B.Tech Civil": 100000
        },
        "hostel_fees": {
            "non-ac": {
                "4-sharing": 95000,
                "3-sharing": 100000,
                "2-sharing": 105000,
                "single-sharing": 110000
            },
            "ac": {
                "4-sharing": 110000,
                "3-sharing": 115000,
                "2-sharing": 120000,
                "single-sharing": 125000
            }
        },
        "base_admission_fee": 3000,
        "admin_contacts": {
            "Admission Office": "9876543210",
            "Student Help Desk": "9123456780",
            "Email": "admissions@jaiuniversity.edu"
        }
    }

In [15]:
def calculate_admission_fee(percentage, base_fee):
    return 0 if percentage >= 80 else base_fee

In [16]:
def select_hostel(hostel_data):
    hostel_type = input("Choose hostel type (AC / Non-AC): ").lower()
    if hostel_type not in hostel_data:
        hostel_type = "non-ac"

    print("\nAvailable Sharing Options:")
    for s, f in hostel_data[hostel_type].items():
        print(f"• {s} : ₹{f}")

    sharing = input("Choose sharing type: ").lower()
    if sharing not in hostel_data[hostel_type]:
        sharing = "4-sharing"

    return hostel_type.upper(), sharing, hostel_data[hostel_type][sharing]

In [17]:
def get_student_details():
    print("\n--- Student Details ---")
    name = input("Student Name: ")
    dob = input("Date of Birth (DD-MM-YYYY): ")
    percentage = float(input("12th Percentage: "))
    return name, dob, percentage

In [18]:
def generate_bill(course, course_fee, hostel_fee, admission_fee):
    total = course_fee + hostel_fee + admission_fee

    print("\n========== ADMISSION BILL ==========")
    print(f"Course Fee    : ₹{course_fee}")
    print(f"Hostel Fee    : ₹{hostel_fee}")
    print(f"Admission Fee : ₹{admission_fee}")
    print("-----------------------------------")
    print(f"TOTAL         : ₹{total}")
    print("===================================")

In [19]:
def slot_booking_receipt(name, course):
    print("\n🎉 ADMISSION CONFIRMED 🎉")
    print("------ SLOT BOOKING RECEIPT ------")
    print(f"Student Name : {name}")
    print(f"Course       : {course}")
    print("Slot Status  : BOOKED")
    print("Reporting To : Admission Office")
    print("---------------------------------")

In [20]:
def chatbot():
    data = load_university_data()

    intro = f"Welcome to {data['university_name']}. You are an admission assistant chatbot."
    print(llm_response(intro))

    print("\nEngineering Courses Available:")
    for c in data["engineering_courses"]:
        print(f"• {c}")

    course = input("\nSelect a course: ")
    if course not in data["engineering_courses"]:
        print("Invalid course.")
        return

    course_fee = data["engineering_courses"][course]

    hostel_required = input("Do you need hostel? (yes/no): ").lower()
    hostel_fee = 0

    if hostel_required == "yes":
        h_type, sharing, hostel_fee = select_hostel(data["hostel_fees"])
        print(llm_response(f"User selected {h_type} hostel with {sharing}."))

    print("\nAdmission Office Contacts:")
    for d, c in data["admin_contacts"].items():
        print(f"{d}: {c}")

    proceed = input("\nProceed with admission booking? (yes/no): ").lower()
    if proceed != "yes":
        print("Admission cancelled.")
        return

    name, dob, percentage = get_student_details()
    admission_fee = calculate_admission_fee(percentage, data["base_admission_fee"])

    if admission_fee == 0:
        print(llm_response("Congratulate the student for scoring above 80 percent and getting free admission."))
    else:
        print(f"Admission Fee: ₹{admission_fee}")

    generate_bill(course, course_fee, hostel_fee, admission_fee)
    slot_booking_receipt(name, course)

In [ ]:
chatbot()

Welcome to JAI University. You are an admission assistant chatbot.c that walked through an academic year long, and it must take 3-6 minutes for the student life on one person or whatever school has something going do

Engineering Courses Available:
• B.Tech Computer Science
• B.Tech Artificial Intelligence
• B.Tech Data Science
• B.Tech Cyber Security
• B.Tech Electronics
• B.Tech Mechanical
• B.Tech Civil

Select a course: B.Tech Computer Science
Do you need hostel? (yes/no): yes
Choose hostel type (AC / Non-AC): ac

Available Sharing Options:
• 4-sharing : ₹110000
• 3-sharing : ₹115000
• 2-sharing : ₹120000
• single-sharing : ₹125000
Choose sharing type: 4-sharing
User selected AC hostel with 4-sharing.it.

Admission Office Contacts:
Admission Office: 9876543210
Student Help Desk: 9123456780
Email: admissions@jaiuniversity.edu

Proceed with admission booking? (yes/no): yes

--- Student Details ---
Student Name: Lohith kumar
